## 요약



In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 그래프의 폰트 출력을 선명하게 (svg, retina 등이 있음)
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("default")
# 이미지가 레이아웃 안으로 들어오도록 함
# https://matplotlib.org/stable/users/explain/axes/constrainedlayout_guide.html
mpl.rcParams.update({"figure.constrained_layout.use": True})

#font, line, marker 등의 배율 설정: paper, notebook, talk, poster
sns.set_context("paper") 
#배색 설정: tab10, Set2, Accent, husl
sns.set_palette("Set2") 
#눈금, 배경, 격자 설정: ticks, white, whitegrid, dark, darkgrid
# withegrid: 눈금을 그리고, 각 축의 눈금을 제거
sns.set_style("whitegrid") 

In [3]:
# 로컬에서 

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [4]:
# 필요한 패키지 import
from bs4 import BeautifulSoup
import requests
import re
import os, natsort

In [5]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_usedbook'
save_dir = 'processed'
date = 240711
file_name = f'unused_filtered_{date}.csv'

### 앞선 내용 요약
- 비중고도서 목록과 중고도서 목록, 2 파일을 관리하기로 결정
    - 비중고도서 ItemId를 기준으로 각 중고정보 페이지를 접근하여 크롤링
        - 비중고도서 페이지 url
            - ```https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={}```
        - 중고정보 페이지 url
            -```https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId={}&TabType=3&Fix=1``` 
        - ItemId에 규칙이 따로 없는 것으로 추정
    - 비중고도서 목록에는 각 도서별 정보 (ItemId, 도서명, 저자, 출판사, 세일즈포인트 등)
    - 중고도서 목록에는 각 매물 별 정보 (책이름, ItemId, 중고등급, 판매가, 할인율)
- 비중고도서 목록
    - 구분 : '국내도서' 중 저자, 출판사, 대분류가 null이 아닌 경우만 추려서 사용
    - ItemId 기준 중복 정리
    - ItemId 기준 keep=last로 해서 중복 제거
    - 부정확할 수 있는 정보는 이후 크롤링해서 체크 후 업데이트 하는 것으로
        - 저자가 복수일 때 양식이 통일되지 않음
            - 다음 두 양식이 혼용
            1. AA, BB, CC 지음
            2. AA 외 지음
        - SalesPoint는 이후에는 crawling해서 확인하는 걸로
            - 매일 업데이트 되기 때문에 도서별 정보가 다소 다를 수 있음

**정해진 프로세스**

0. 임의의 연/월/주를 10개 골라서 itemid 목록 작성
1. 임의의 새 상품 페이지를 들어간다
2. 새 상품 관련 정보 수집
3. 해당 도서의 중고 정보 페이지로 접근
4. 각 도서별 중고 상품 정보 수집

In [36]:
dir_path = os.path.join(PRJCT_PATH,'samples')
files = os.listdir(dir_path)
files = natsort.natsorted(files)

df_dict = dict()
for file in files:
    if file[-3:] != 'csv' : continue
    info = file.split('_')[0]
    file_path = os.path.join(dir_path,file)
    df_temp = pd.read_csv(file_path)
    df_temp['source'] = info
    df_dict[info] = df_temp
    

In [ ]:
# 가져올 책 범위,ID.. 값 정의
start_id = 97043285 # 125190808
end_id = 97046595





In [ ]:
# ID를 증가시키며 책 정보 크롤링
for book_id in range(start_id, end_id + 1):
    url = f'http://www.yes24.com/Product/Goods/{book_id}'
    try:
        r = requests.get (url)
        if r.status_code == 200:
            html=r.text
            soup=BeautifulSoup(html, 'lxml') #  BeautifulSoup 클래스의 인스턴스 생성

            # title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2').get_text()
            try:
                title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2').get_text()
            except AttributeError:
                title = None # 선택한 요소가 없어서 발생하는 AttributeError 처리

            try:
                rating = soup.select_one('#spanGdRating > a > em').get_text() # .get_text().strip()
            except AttributeError:
                rating = None

            try:
                author = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a').get_text()
            except AttributeError:
                author = None

            btype = 'new'
            try:
                # if soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > strong > em').get_text() == "중고도서":
                if "중고도서" in soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > strong > em').get_text().lower():
                    price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(1) > table > tbody > tr.accentRow > td > span > em').get_text()  # 판매가격
                    btype = 'old'
                else:
                    price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(3) > table > tbody > tr:nth-child(2) > td > span > em').get_text()  # 판매가격
            except AttributeError:
                price = None

            try:
                pdate = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date').get_text()   # 발행일
            except AttributeError:
                pdate = None

            try:
                publisher = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a').get_text()  # 출판사
            except AttributeError:
                publisher = None

            try:
                numbers = re.findall(r'\d+', soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_sellNum').get_text().replace(',', ''))
                number = int(numbers[0]) if numbers else None
                salseindex = number # soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_sellNum').get_text()  # 판매지수
            except AttributeError:
                salseindex = None



            # book_df = book_df.append({'ID':book_id, 'Title':title, 'Author':author, 'Rating':rating, 'Price':price, 'Pdate':pdate, 'Publisher':publisher, 'Btype':btype}, ignore_index=True)
            new_row = pd.DataFrame({'ID': [book_id], 'Title': [title], 'Author': [author], 'Rating': [rating], 'Price': [price], 'Pdate': [pdate], 'Publisher': [publisher], 'Btype': [btype], 'Salseindex': [salseindex]})
            book_df = pd.concat([book_df, new_row], ignore_index=True)

        else:
            print('에러발생1')
    except requests.RequestException as e:
        # 에러 발생시....
        print('에러발생2')
# book_df
book_df.to_csv('book.csv', encoding='euc-kr')
